# Gurgaon Real Estate Data Analysis

This notebook performs:
- Data loading
- Exploration and visualization
- Missing values handling
- Outlier detection and trimming

Dataset: `../data/Gurgaon_RealEstate.csv`

In [2]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Load Dataset
df = pd.read_csv('../data/Gurgaon_RealEstate.csv')
df.head()

In [ ]:
# Inspect Data
df.info()

In [ ]:
# Remove Duplicates
df.drop_duplicates(inplace=True)

## Property Type Distribution

In [ ]:
plt.figure(figsize=(10,6))
sns.countplot(x='property_type', data=df)
plt.title('Property Type Distribution')
plt.show()

## Remove Societies with Less Than 3 Records

In [ ]:
society_freq = df['society'].value_counts()
societies_to_remove = society_freq[society_freq < 3].index
df = df[~df['society'].isin(societies_to_remove)]

## Price Statistics and Plots

In [ ]:
print("Price Summary Statistics:")
print(df['price'].describe())

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(df['price'], kde=True)
plt.title('Price Histogram')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(y='price', data=df)
plt.title('Price Boxplot')
plt.show()

In [ ]:
print("Price Skewness:", df['price'].skew())
print("Price Kurtosis:", df['price'].kurt())

## Bathroom Statistics

In [ ]:
print("Bathroom Summary Statistics:")
print(df['bathroom'].describe())

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(df['bathroom'], kde=True)
plt.title('Bathroom Histogram')
plt.show()

## Scatter and Box Plots

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x='area', y='price', data=df)
plt.title('Price vs Area')
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='property_type', y='price', data=df)
plt.title('Property Type vs Price')
plt.show()

## Missing Values

In [ ]:
print("Missing Values Distribution:")
print(df.isnull().sum())

## KNN Imputation

In [ ]:
from sklearn.impute import KNNImputer
numeric_cols = df.select_dtypes(include='number').columns
knn = KNNImputer(n_neighbors=5)
for col in numeric_cols:
    df[col] = knn.fit_transform(df[[col]])

## Encode 'facing'

In [ ]:
df['facing'] = df['facing'].astype('category').cat.codes

## Fill 'society' with Mode

In [ ]:
mode_society = df['society'].mode()[0]
df['society'] = df['society'].fillna(mode_society)

## Outlier Detection

In [ ]:
outliers_z = []
outliers_iqr = []
for col in numeric_cols:
    z_scores = np.abs((df[col] - df[col].mean()) / df[col].std())
    outliers_z.extend(df[z_scores > 3].index)

    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    outliers_iqr.extend(df[(df[col] < Q1 - 1.5*IQR) | (df[col] > Q3 + 1.5*IQR)].index)

print("Outliers Z-score:", len(set(outliers_z)))
print("Outliers IQR:", len(set(outliers_iqr)))

## Trimming Outliers

In [ ]:
Q1 = df[numeric_cols].quantile(0.15)
Q3 = df[numeric_cols].quantile(0.85)
IQR = Q3 - Q1
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR
df_trimmed = df.copy()
for col in numeric_cols:
    df_trimmed = df_trimmed[(df_trimmed[col] >= lower[col]) & (df_trimmed[col] <= upper[col])]

## Histograms After Trimming

In [ ]:
df_trimmed.hist(figsize=(12,10))
plt.tight_layout()
plt.show()